In [ ]:
import pyogrio

def shapefile2fingerprint(shapefile: str|gpd.GeoDataFrame, buffer:int=0, proj: str|int=3857) -> xr.DataArray:
    if isinstance(shapefile, str):
        shapefile_gpd = pyogrio.read_dataframe(shapefile)
    elif isinstance(shapefile, gpd.GeoDataFrame):
        shapefile_gpd = shapefile
    
    if buffer > 0:
        shapefile_gpd = shapefile_gpd.to_crs(proj).buffer(buffer*1e3).to_crs(4326)
    
    grid_gpd = pyogrio.read_dataframe(grid_fn)
    
    # Compute the intersection between the fingerprint and the grid
    grid_clip_gpd = gpd.clip(grid_gpd, shapefile_gpd)
    grid_clip_gpd['area'] = grid_clip_gpd.to_crs(proj).area
    
    grid_gpd['ratio'] = 0
    grid_gpd['area'] = grid_gpd.to_crs(proj).area

    grid_gpd.loc[grid_clip_gpd.index, 'ratio'] = grid_clip_gpd['area'] / grid_gpd['area']
    fingerprint_xr = grid_gpd.loc[:, ['lat','lon','ratio']].reset_index(drop=True).set_index(['lat', 'lon']).to_xarray().sortby('lon', ascending=True).sortby('lat', ascending=False).ratio
    return fingerprint_xr

In [ ]:
from processing_functions import *
from importResults import missing_month, _X_grace_END_GRID_MEAN_NoGIA, _X_mascons_CSR, _X_mascons_JPL
import re

here_dir = os.path.abspath(os.getcwd())

ddk_filter_file = os.path.join(model_dir, "2_Filtre_DDK", "DDK7.ddk")
le_filter_file = os.path.join(model_dir, "1_Filtre_LE", "coefficients_LE_FILTER_DDK7.coef")
love_number_file = os.path.join(model_dir, "3_Load_Love", "Load_Love2_CF_deg0.dat")

trend_xr = grid_xr2Poly2nSin(_X_grace_END_GRID_MEAN_NoGIA).trend

Leakage Continental & Océanique

In [ ]:
ddk_filter_file = os.path.join(model_dir, "2_Filtre_DDK", "DDK7.ddk")
le_filter_file = os.path.join(model_dir, "1_Filtre_LE", "coefficients_LE_FILTER_DDK7.coef")
save_map(trend_xr, vmin=-5, vmax=5, title="GRACE/-FO trend", show=True, noTitle=False, byYear=True, engine="pygmt")
ocean_signal = 0.2  # cm/an
ocean_xr = (get_oceans() > 0.7) * 1
mask_non_oceanic_signal = -ocean_xr + 1 + (abs(trend_xr * ocean_xr) - 0.8 > 0) * 1
mask_oceanic_signal = -mask_non_oceanic_signal + 1
save_map(mask_non_oceanic_signal, vmin=0, vmax=1, title="Masque Océan / Continents(+séismes)", show=True, noTitle=False, byYear=True)
oceanic_signal = mask_oceanic_signal * ocean_xr
oceanic_signal_filtered = grace_df2grid_xr(
    grace_df_LOBE_EDGE(
        grace_dfDecorelation(grid_xr2grace_df(oceanic_signal, isGCoeff=True), file_ddk_filter=ddk_filter_file), le_filter_file=le_filter_file
    ),
    isEWH=True,
)
oceanic_leakage = (oceanic_signal_filtered - oceanic_signal) * (-ocean_xr + 1)
save_map(oceanic_leakage, vmin=-1, vmax=1, title="Oceanic Leakage", show=True, noTitle=False, byYear=True)
continental_signal = (-ocean_xr + 1) * trend_xr
continental_signal_filtered = grace_df2grid_xr(
    grace_df_LOBE_EDGE(
        grace_dfDecorelation(grid_xr2grace_df(continental_signal, isGCoeff=True), file_ddk_filter=ddk_filter_file), le_filter_file=le_filter_file
    ),
    isEWH=True,
)
contiental_leakage = (continental_signal_filtered - continental_signal) * ocean_xr
save_map(contiental_leakage, vmin=-1, vmax=1, title="Continental Leakage", show=True, noTitle=False, byYear=True)
save_map(contiental_leakage + oceanic_leakage, vmin=-1, vmax=1, title="Oceanic & Continental Leakage", show=True, noTitle=False, byYear=True)
save_map(
    trend_xr - (contiental_leakage + oceanic_leakage),
    vmin=-5,
    vmax=5,
    title="GRACE/-FO trend corrected for Oceanic & Continental Leakage",
    show=True,
    noTitle=False,
    byYear=True,
)

Signal océanique à considérer

In [ ]:
ocean_signal = 0.2  # cm/an
ocean_xr = (get_oceans() > 0.7) * 1

mask_non_oceanic_signal = -ocean_xr + 1 + (abs(trend_xr * ocean_xr) - 0.8 > 0) * 1
mask_oceanic_signal = -mask_non_oceanic_signal + 1

save_map(mask_non_oceanic_signal, show=True, engine="cartopy")
save_map(trend_xr * mask_non_oceanic_signal, cmap=cmap_5, show=True, engine="pygmt")

dates = []
for year in range(2000, 2030):
    for month in range(1, 13):
        dates.append(pda.to_datetime("{:4d}-{:02d}-15".format(year, month), format="%Y-%m-%d"))
ocean_signal_xr = []
for date in tqdm(dates, desc="Etalement sur le temps"):
    g_date = gregorianDate2decimalYear(pda.to_datetime(date))
    ocean_signal_xr.append(g_date * mask_oceanic_signal * ocean_signal)
ocean_signal_xr = xr.concat(ocean_signal_xr, dates).rename({"concat_dim": "date"})
ocean_signal_xr = grid_xr2rem_time_mean_xr(ocean_signal_xr)


print("###################    1ère méthode    ###################")

non_oceanic_signal = _X_grace_END_GRID_MEAN_NoGIA * mask_non_oceanic_signal + ocean_signal_xr * mask_oceanic_signal
print(f"Signal continental + signal océanique artificiel {ocean_signal}cm/an")
save_map(grid_xr2Poly2nSin(non_oceanic_signal).trend, cmap=cmap_5, engine="pygmt", show=True)

non_oceanic_signal = grid_xr2rem_time_mean_xr(
    grace_df2grid_xr(
        grace_df_LOBE_EDGE(grace_dfDecorelation(grid_xr2grace_df(non_oceanic_signal, isGCoeff=True), ddk_filter_file), le_filter_file), isEWH=True
    )
)
print(f"Signal continental + signal océanique artificiel {ocean_signal}cm/an filtré avec DDK7 + LE")
save_map(grid_xr2Poly2nSin(non_oceanic_signal).trend, cmap=cmap_5, engine="pygmt", show=True)

non_oceanic_signal = non_oceanic_signal - ocean_signal_xr * mask_oceanic_signal - _X_grace_END_GRID_MEAN_NoGIA * (-ocean_xr + 1)
print(f"Signaux résiduels dans les océans non issus des océans")
save_map(grid_xr2Poly2nSin(non_oceanic_signal).trend, cmap=cmap_5, engine="pygmt", show=True)

print(f"Données corrigées des signaux non océaniques (leakage + EQ)")
oceanic_signal_xr = _X_grace_END_GRID_MEAN_NoGIA - non_oceanic_signal
save_map(grid_xr2Poly2nSin(oceanic_signal_xr).trend, cmap=cmap_5, engine="pygmt", show=True)

write_netCDF(oceanic_signal_xr, os.path.join(here_dir, "oceanic_signal_xr.nc"))

In [ ]:
ocean_signal = 0.2  # cm/an
ocean_xr = (get_oceans() > 0.7) * 1

mask_non_oceanic_signal = -ocean_xr + 1 + (abs(trend_xr * ocean_xr) - 0.8 > 0) * 1
mask_oceanic_signal = -mask_non_oceanic_signal + 1

save_map(mask_non_oceanic_signal, show=True, engine="cartopy")
save_map(trend_xr * mask_non_oceanic_signal, cmap=cmap_5, show=True, engine="pygmt")

dates = []
for year in range(2000, 2030):
    for month in range(1, 13):
        dates.append(pda.to_datetime("{:4d}-{:02d}-15".format(year, month), format="%Y-%m-%d"))
ocean_signal_xr = []
for date in tqdm(dates, desc="Etalement sur le temps"):
    g_date = gregorianDate2decimalYear(pda.to_datetime(date))
    ocean_signal_xr.append(g_date * mask_oceanic_signal * ocean_signal)
ocean_signal_xr = xr.concat(ocean_signal_xr, dates).rename({"concat_dim": "date"})
ocean_signal_xr = grid_xr2rem_time_mean_xr(ocean_signal_xr)

print("###################    2ème méthode    ###################")
non_oceanic_signal = _X_grace_END_GRID_MEAN_NoGIA * mask_non_oceanic_signal + ocean_signal_xr * mask_oceanic_signal
save_map(grid_xr2Poly2nSin(non_oceanic_signal).trend, cmap=cmap_5, engine="pygmt", show=True)

non_oceanic_signal = grid_xr2rem_time_mean_xr(
    grace_df2grid_xr(
        grace_df_LOBE_EDGE(
            grace_dfDecorelation(grid_xr2grace_df(non_oceanic_signal, love_number_file=love_number_file), ddk_filter_file), le_filter_file
        ),
        love_number_file=love_number_file,
    )
)
save_map(grid_xr2Poly2nSin(non_oceanic_signal).trend, cmap=cmap_5, engine="pygmt", show=True)
non_oceanic_signal = non_oceanic_signal - ocean_signal_xr * mask_oceanic_signal - _X_grace_END_GRID_MEAN_NoGIA * (-ocean_xr + 1)
save_map(grid_xr2Poly2nSin(non_oceanic_signal).trend, cmap=cmap_5, engine="pygmt", show=True)

oceanic_signal_xr = _X_grace_END_GRID_MEAN_NoGIA - non_oceanic_signal
save_map(grid_xr2Poly2nSin(oceanic_signal_xr).trend, cmap=cmap_5, engine="pygmt", show=True)

write_netCDF(oceanic_signal_xr, os.path.join(here_dir, "oceanic_signal_xr_deg0.nc"))

Export des trends océaniques

In [ ]:
TS_files = list(filter(lambda x: "TS" in x and "oc_buffer" in x and "oceanic_signal" in x and "lat65" not in x, os.listdir(here_dir)))

data = {}
for TS_file in TS_files:
    (solution, buffer) = re.findall(r"TS_(\S+)_oc_buffer_(\d*)km.pickle", TS_file)[0]
    buffer = int(buffer)
    if solution not in data:
        data[solution] = {}
    data[solution][buffer] = read_pickle(TS_file)

for solution in data.keys():
    date_start = pda.to_datetime("2005-01-01")
    date_end = pda.to_datetime("2018-12-31")

    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    for buffer in [50, 100, 200, 500, 700, 1000]:
        period = (data[solution][buffer].date >= date_start) & (data[solution][buffer].date < date_end)
        # period = (data[solution][buffer].date>=pda.to_datetime("2005-01-01")) & (data[solution][buffer].date<pda.to_datetime("2015-12-31"))

        data_t = data[solution][buffer]
        TS = data_t.values
        DT = data_t.date.values
        DY = np.array(list(map(gregorianDate2decimalYear, DT)))

        (p0, p1, p2, p3, a1, phi1, a2, phi2) = model_8p(TS, DT)
        print("Buffer : {} km".format(buffer))
        print("p1 : {:.02f} mm/yr".format(p1 * 10))
        print("p2 : {:.02f} mm/yr²\n".format(p2 * 10))

        signal_to_remove = a1 * np.cos(2 * np.pi * DY - phi1 * np.pi / 6) + a2 * np.cos(4 * np.pi * DY - phi2 * np.pi / 3)

        TS = (TS - signal_to_remove) * 10  # Retrait du signal plus conversion en mm
        TS -= np.mean(TS[:6])

        period_index = ((DT >= date_start) * 1 * (DT <= date_end) * 1) == 1
        (p0, p1) = model_2p(TS[period_index], DT[period_index])
        if buffer == 700:
            fit_values = p0 + p1 * DY
            plt.plot(DT, fit_values, label="Linear fit (700 km) \n a={:.02f}mm/an b={:.02f}mm".format(p1, p0))
        label = "{} km - ".format(buffer) + "$\dot{\sigma}$" + " = {:.02f} mm/yr".format(p1)

        plt.plot(DT, TS, label=label)

    # TODO : Supprimer les signaux liés aux séismes !

    plt.xlim((np.min(DT), np.max(DT)))
    plt.ylim((-2, 40))
    ylim = plt.ylim()
    for i, period in enumerate(missing_month):
        date_period_min = min(period)
        date_period_max = max(period)

        if i == 0:
            plt.fill_betweenx(ylim, date_period_min, date_period_max, alpha=0.3, zorder=-1, color="C0", label="Missing data", edgecolor=None)
        else:
            plt.fill_betweenx(ylim, date_period_min, date_period_max, alpha=0.3, zorder=-1, color="C0", edgecolor=None)

    # plt.title('Sum of the signals over the whole oceans depending of the buffer to the coast')
    plt.xlabel("Time")
    plt.ylabel("mean SLE [mm]")
    plt.title("Period : {:02d}-{:4d} - {:02d}-{:4d}".format(date_start.month, date_start.year, date_end.month, date_end.year))
    plt.legend()
    plt.savefig("mean_SLR_{}.png".format(solution), dpi=600)
    plt.show()